In [7]:
from sqlalchemy import create_engine, text, inspect
import pandas as pd
db_path = r'C:\Users\vivia\co-pilot-v1\Notebooks\bulk_upload.db'

engine = create_engine(f'sqlite:///{db_path}')
# Create an inspector
inspector = inspect(engine)

# Get the list of table names
tables = inspector.get_table_names()

# Print the structure of each table
for table in tables:
    print(f"\nTable: {table}")
    columns = inspector.get_columns(table)
    for column in columns:
        print(f"  Column: {column['name']} - {column['type']}")

# Additionally, print foreign key information
for table in tables:
    print(f"\nForeign Keys for Table: {table}")
    foreign_keys = inspector.get_foreign_keys(table)
    for fk in foreign_keys:
        print(f"  Foreign Key: {fk['constrained_columns']} -> {fk['referred_table']}.{fk['referred_columns']}")

with engine.connect() as connection:
    query = text("""SELECT p.part_number, m.model_name, p.price, p.quantity, p.make, p.description, p.months_no_sale, p.value, y.year, p.uploaded
                    FROM model_year my
                    JOIN models m ON my.model_id=m.id
                    JOIN parts p ON my.part_number=p.part_number
                    JOIN years y ON my.year_id=y.id
                    WHERE LOWER(p.make)='honda' and LOWER(m.model_name)='pilot'
                 """)
    df = connection.execute(query)
df = pd.DataFrame(df.fetchall())
df


Table: model_year
  Column: id - INTEGER
  Column: part_number - INTEGER
  Column: model_id - INTEGER
  Column: year_id - INTEGER

Table: models
  Column: id - INTEGER
  Column: part_number - VARCHAR
  Column: model_name - VARCHAR

Table: parts
  Column: part_number - VARCHAR
  Column: description - VARCHAR
  Column: quantity - INTEGER
  Column: value - FLOAT
  Column: price - FLOAT
  Column: make - VARCHAR
  Column: months_no_sale - INTEGER
  Column: uploaded - BOOLEAN

Table: years
  Column: id - INTEGER
  Column: part_number - VARCHAR
  Column: year - INTEGER

Foreign Keys for Table: model_year
  Foreign Key: ['part_number'] -> parts.['part_number']
  Foreign Key: ['model_id'] -> models.['id']
  Foreign Key: ['year_id'] -> years.['id']

Foreign Keys for Table: models
  Foreign Key: ['part_number'] -> parts.['part_number']

Foreign Keys for Table: parts

Foreign Keys for Table: years
  Foreign Key: ['part_number'] -> parts.['part_number']


,part_number,model_name,price,quantity,make,description,months_no_sale,value,year,uploaded
0,94109-14000,Pilot,0.37,534,Honda,WASHER DRAIN 14MM,0,197.58,2022,1
1,94109-14000,Pilot,0.37,534,Honda,WASHER DRAIN 14MM,0,197.58,2021,1
2,OGIO,Pilot,38.27,80,Honda,HONDA GOLF SHIRT,1,3061.60,2022,1
3,OGIO,Pilot,38.27,80,Honda,HONDA GOLF SHIRT,1,3061.60,2021,1
4,OGIO,Pilot,38.27,80,Honda,HONDA GOLF SHIRT,1,3061.60,2023,1
...,...,...,...,...,...,...,...,...,...,...
1830,PH2,Pilot,28.62,1,Honda,OIL FILTER,8,3.29,2022,0
1831,PH2,Pilot,28.62,1,Honda,OIL FILTER,8,3.29,2021,0
1832,PH2,Pilot,28.62,1,Honda,OIL FILTER,8,3.29,2020,0
1833,TUCHN-H87-7PA1,Pilot,12.76,1,Honda,T/U PAINT,11,10.75,2023,0
